# Prepare environment
Load libraries, etc.

In [1]:
%pip install -q transformers librosa datasets==2.14.6 evaluate jiwer gradio bitsandbytes==0.37 accelerate geomloss
%pip install -q git+https://github.com/huggingface/peft.git@main

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from huggingface_hub import notebook_login

notebook_login() #huggingface-cli login workaround

In [40]:
from datasets import load_dataset, DatasetDict
from transformers import (WhisperFeatureExtractor, 
                          WhisperTokenizer, 
                          WhisperProcessor,
                          WhisperModel,
                          WhisperForConditionalGeneration, 
                          Seq2SeqTrainingArguments, 
                          Seq2SeqTrainer, 
                          TrainerCallback, 
                          TrainingArguments, 
                          TrainerState, 
                          TrainerControl)
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from geomloss import SamplesLoss
from peft import (prepare_model_for_int8_training,
                  LoraConfig, 
                  PeftModel, 
                  LoraModel, 
                  LoraConfig, 
                  TaskType,
                  get_peft_model)
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
import re


# Load dataset

In [5]:
sd_qa = DatasetDict()

sd_qa["dev"] = load_dataset("WillHeld/SD-QA", split="dev", token=True)
sd_qa["test"] = load_dataset("WillHeld/SD-QA", split="test", token=True)

/Applications/miniconda3/envs/cs224n-project-env/lib/python3.9/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [6]:
saved_data = sd_qa
print(sd_qa)
print(sd_qa['dev'])

DatasetDict({
    dev: Dataset({
        features: ['id', 'aus', 'gbr', 'ind_n', 'ind_s', 'irl', 'kenya', 'nga', 'nzl', 'phl', 'usa', 'zaf', 'answers', 'question'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['id', 'aus', 'gbr', 'ind_n', 'ind_s', 'irl', 'kenya', 'nga', 'nzl', 'phl', 'usa', 'zaf', 'answers', 'question'],
        num_rows: 1031
    })
})
Dataset({
    features: ['id', 'aus', 'gbr', 'ind_n', 'ind_s', 'irl', 'kenya', 'nga', 'nzl', 'phl', 'usa', 'zaf', 'answers', 'question'],
    num_rows: 1000
})


# Prepare data

In [7]:
# select only target and source dialect
target_dialect = 'usa'
source_dialect = 'ind_n'
sd_qa = sd_qa.select_columns(['id', source_dialect, target_dialect])
print(sd_qa['dev'][0])

{'id': '-1008642825401516622', 'ind_n': {'path': None, 'array': array([ 0.00000000e+00, -3.05175781e-05, -3.05175781e-05, ...,
        3.96728516e-04,  2.13623047e-04,  6.10351562e-05]), 'sampling_rate': 16000}, 'usa': {'path': None, 'array': array([0.        , 0.        , 0.        , ..., 0.00201416, 0.00259399,
       0.00262451]), 'sampling_rate': 16000}}


In [8]:
# load whisper feature extractor, tokenizer, processor
model_path = "openai/whisper-base"
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_path)
task = "transcribe"
tokenizer = WhisperTokenizer.from_pretrained(model_path, task=task)
processor = WhisperProcessor.from_pretrained(model_path, task=task)

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
# define prepare dataset function to extract audio features
def prepare_dataset(batch):
    # compute log-Mel input features from input audio array
    batch["source_input_features"] = feature_extractor(batch[source_dialect]["array"], sampling_rate=batch[source_dialect]["sampling_rate"]).input_features[0]
    batch["target_input_features"] = feature_extractor(batch[target_dialect]["array"], sampling_rate=batch[target_dialect]["sampling_rate"]).input_features[0]
    return batch

In [10]:
# map feature extracter to sd_qa
# sd_qa = sd_qa.map(prepare_dataset, remove_columns=sd_qa.column_names["dev"], num_proc=2)
sd_qa = sd_qa.map(prepare_dataset, remove_columns=[source_dialect, target_dialect], num_proc=2)

Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

/Applications/miniconda3/envs/cs224n-project-env/lib/python3.9/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/Applications/miniconda3/envs/cs224n-project-env/lib/python3.9/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/Applications/miniconda3/envs/cs224n-project-env/lib/python3.9/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Map (num_proc=2):   0%|          | 0/1031 [00:00<?, ? examples/s]

/Applications/miniconda3/envs/cs224n-project-env/lib/python3.9/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/Applications/miniconda3/envs/cs224n-project-env/lib/python3.9/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)


In [11]:
print(sd_qa['dev'])

Dataset({
    features: ['id', 'source_input_features', 'target_input_features'],
    num_rows: 1000
})


# Training 

In [12]:
# Define a data collator
@dataclass
class DataCollatorSpeechSeq2Seq:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # convert source inputs to pytorch tensors
        source_input_features = [{"source_input_features": feature["source_input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(source_input_features, return_tensors="pt")

        # convert target inputs to pytorch tensors
        target_input_features = [{"target_input_features": feature["target_input_features"]} for feature in features]
        batch["target_input_features"] = self.processor.feature_extractor.pad(target_input_features, return_tensors="pt")["target_input_features"]

        return batch

# Initialize a data collator
data_collator = DataCollatorSpeechSeq2Seq(processor=processor)

In [ ]:
# Define evaluation metrics
sinkhorn_loss = SamplesLoss(loss="sinkhorn", p=2)

# Define a function to compute metrics
def compute_metrics(pred):
    loss = sinkhorn_loss(pred.predictions, pred.target)
    return {"loss": loss}

In [58]:
# example of generating last encoder hidden state
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")

example = feature_extractor(saved_data['dev'][0]['aus']['array'], return_tensors="pt", sampling_rate=16000)
decoder_input_ids = torch.tensor([[1, 1]]) * model.config.decoder_start_token_id

with torch.no_grad():
  outputs = model(example.input_features, decoder_input_ids=decoder_input_ids, output_hidden_states=True)

last_hidden_state = outputs.encoder_hidden_states[-1]
print(last_hidden_state)

tensor([[[-1.4458, -0.3243, -0.9025,  ..., -1.2311,  0.6060,  0.0106],
         [-0.6728, -0.6361,  0.7436,  ..., -0.1701, -0.1883, -0.3665],
         [-0.0632, -0.8393,  0.6713,  ...,  0.2727, -0.0833, -0.2981],
         ...,
         [-0.4062, -1.6662,  0.4940,  ...,  0.7445, -0.3753,  0.6327],
         [-0.7755, -1.5057,  0.4120,  ...,  0.4407,  0.0104,  0.1389],
         [-1.3152, -1.0738,  0.9730,  ...,  0.6352, -1.7935, -0.9234]]])


In [13]:
# Load pre-trained checkpoint in 8b
model = WhisperForConditionalGeneration.from_pretrained(model_path,load_in_8bit=True, device_map="auto")

# Post-processing steps on the model
model = prepare_model_for_int8_training(model, output_embedding_layer_name="proj_out")

# Make inputs trainable
def make_inputs_require_grad(module, input, output):
    output.requires_grad_(True)

model.model.encoder.conv1.register_forward_hook(make_inputs_require_grad)

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

RuntimeError: No GPU found. A GPU is needed for quantization.

In [60]:
print(model)
def get_target_modules(model):
    model_modules = str(model.modules)
    pattern = r'\((\w+)\): Linear'
    linear_layer_names = re.findall(pattern, model_modules)

    names = []
    # Print the names of the Linear layers
    for name in linear_layer_names:
        names.append(name)
    target_modules = list(set(names))

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 512, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 512)
      (layers): ModuleList(
        (0-5): 6 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=False)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          

In [ ]:
# Apply LoRA to model, targeting all layers of encoder
target_modules = ['k_proj', 'v_proj', 'q_proj', 'out_proj', 'fc1', 'fc2']
config = LoraConfig(r=32, # rank, adjust this
                    lora_alpha=64, 
                    target_modules = target_modules, 
                    lora_dropout=0.05, 
                    bias="none",
                    task_type=TaskType.FEATURE_EXTRACTION,
                    ) # task_type= ????? 

model = get_peft_model(model, config)
model.print_trainable_parameters()

# Define training configuration
training_args = Seq2SeqTrainingArguments(
    output_dir="reach-vb/test",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-3,
    warmup_steps=50,
    num_train_epochs=3,
    evaluation_strategy="steps",
    fp16=True,
    per_device_eval_batch_size=8,
    generation_max_length=128,
    logging_steps=100,
#    max_steps=100, # only for testing purposes, remove this from your final run :)
    remove_unused_columns=False,  # required as the PeftModel forward doesn't have the signature of the wrapped model's forward
    label_names=["labels"],  # same reason as above
)

In [ ]:
# Additional PEFT things
# This callback helps to save only the adapter weights and remove the base model weights.
class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control


trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=sd_qa["dev"],
    eval_dataset=sd_qa["test"],
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
    callbacks=[SavePeftModelCallback],
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [ ]:
# Run train
trainer.train()

In [ ]:
# Save to hub
peft_model_id = "azure-224n/whisper-base-100steps"
model.push_to_hub(peft_model_id)

# Evaluation

# Inference